<a href="https://colab.research.google.com/github/shubham007kumar/INT247_KM058_RKM058a21/blob/master/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install kaggle


In [0]:
from google.colab import files
uploaded = files.upload()


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 as cv
import seaborn as sns

In [0]:
data=pd.read_csv('HAM10000_metadata.csv')

In [0]:
data=data.drop(columns=['lesion_id'])

In [0]:
data.head()

In [0]:

data.isnull().sum()

In [0]:
data=data.dropna()

In [0]:

data.isnull().sum()

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
y=data.dx


In [0]:
y.head()

In [0]:
feature=['image_id','age','sex','dx_type']
x=data[feature]
x.head()


In [0]:
from sklearn.model_selection import train_test_split
train_x,val_x,train_y,val_y=train_test_split(x,y,test_size=0.2)

In [0]:
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
y=lbl.fit_transform(y)
y.shape

In [0]:
from keras.utils import to_categorical

one_hot_train=to_categorical(train_y)
one_hot_train

In [0]:
from keras.utils import to_categorical

one_hot_test=to_categorical(val_y)
one_hot_test

In [0]:

#array reshape
train_x=np.array(train_x).reshape(-1,100,100,3)
train_x=train_x/255.0
val_x=np.array(val_x).reshape(-1,100,100,3)
val_x=val_x/255.0

train_x.shape,val_x.shape,one_hot_train.shape,one_hot_test.shape

In [0]:
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(100,100,3),padding='same'))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Dropout(0.20))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.40))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.20))

model.add(Flatten())

model.add(Dense(64, activation='linear'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(128, activation='linear'))
model.add(Dense(256, activation='linear'))
model.add(Dense(7, activation='softmax'))
model.summary()

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_x,one_hot_train,batch_size=128,epochs=10,validation_split=0.2)

In [0]:

test_loss,test_acc=model.evaluate(val_x,one_hot_test)
test_loss,test_acc